In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os

## Preparing data as Dataframe objecs

In [2]:
path_train_rumors = "train_rumors.json"
path_dev_rumors = "dev_rumors.json"
path_relevance_judgments = "relevance_judgments.txt"

In [3]:
def read_json_file(path):
    f = open(path , encoding="utf8")
    data = json.load(f)
    f.close()
    return data

In [4]:

columns = ["rumor_id", "0", "user_id", "relevance"]
def text_to_dataframe(path , columns):
    df = pd.DataFrame(columns=columns)
    file = open(path , 'r')

    for line in file:
        line = line.strip().split('\t')
        df.loc[len(df)] = line

    file.close()
    
    
    return df

In [5]:
def json_to_dataframe(json_object):
    df = pd.DataFrame()
    for i in json_object[0].keys():
        df[i] = [json_object[j][i] for j in range(len(json_object))]
    return df

The metaadata directory contains **1000 json files**, each one of them contain **1193 lines**. Each line of these 1193 contains **10 features**

## Data analysis

### 1. Train and Developpement rumors

In [6]:
train_rumors = read_json_file(path_train_rumors)
dev_rumors = read_json_file(path_dev_rumors)

train_rumors = json_to_dataframe(train_rumors)
dev_rumors = json_to_dataframe(dev_rumors)

In [7]:
train_rumors.columns , train_rumors.shape 

(Index(['rumor_id', 'tweet_id', 'tweet_text', 'category'], dtype='object'),
 (120, 4))

In [8]:
dev_rumors.columns , dev_rumors.shape

(Index(['rumor_id', 'tweet_id', 'tweet_text', 'category'], dtype='object'),
 (30, 4))

In [9]:
train_rumors.tail(3)

,rumor_id,tweet_id,tweet_text,category
117,AuFIN_118,1425132064450531328,#فيديو || اول مرّة في تاريخ الاندية الفرنسية ن...,sports
118,AuFIN_119,1221841304512057344,شوبير يعترف ان ادارة الأهلي ضغطت على الكاف واج...,sports
119,AuFIN_120,1434275913152217089,عاجل: النايل سات ١٣٠ مليون مشاهد قنوات mbc بر...,sports


In [10]:
dev_rumors.head(3)

,rumor_id,tweet_id,tweet_text,category
0,AuFIN_121,1406642665689649156,بشرى سارة للأشقاء في الخليج …\n\nالمحكمة الإسر...,politics
1,AuFIN_122,1335681592300331010,#عاجل وفاة #خامنئي المرشد الأعلى في إيران منذ ...,politics
2,AuFIN_123,1324220773435727872,#الجزائر ترفض التطبيع العلني وتفضله أن يكون سر...,politics


In [11]:
train_rumors["category"].value_counts()

politics    40
health      40
sports      40
Name: category, dtype: int64

In [12]:
dev_rumors["category"].value_counts()

politics    10
health      10
sports      10
Name: category, dtype: int64

In [13]:
def print_rumors(data):
    for i in data.index:
        print(data["tweet_text"][i])

In [14]:
print_rumors(train_rumors)

موسى أبو مرزوق الحمساوى يهنئ طالبان بفوزه بأفغانستان ويعتبر مندوب لملالي إيران 😉
- الصورة من إجتماع الأرهابيين وفد من طالبان مع هنية ليلة الأمس   😎
تم في كشك قطر "ممول الا.ر.هاب" 
#حفظ_الله_مصر https://t.co/Rnd14ZXv6C
مزيداً من الأنحدار نحو مستنقع التطبيع القذر..حسين الجسمي يشارك في تلحين اغنية إسرائيلية تبدأ برساله شكر للأمارات من الرئيس الأسرائيلي في تل ابيب على أنقاض قرانا الفلسطينية التي مسحت عن وجه الارض...خسئتم انتم عار علينا.. والعروبة والدين بريئةٌ من امثالكم 
#التطبيع_خيانة_عظمى https://t.co/1pdoGqkZSH
بتوجيهات من هيثم بن طارق آل سعيد سلطان عمان استدعت سلطنة عمان  سفيرها في الدوحة، اليوم  ودعت قطر إلى الكف عن نشاطاتها المزعزعة للاستقرار والأمن.
وتأتي الخطوة تعكس التضامن مع المملكة العربية السعودية التي قطعت العلاقات من قطر منتصف عام 2017 ، إضافة إلى الإمارات https://t.co/LMVgSR1OSi
.
.

متظاهرون أتراك يهاجمون البنك الكويتي التركي في إسطنبول .. بعد انهيار الليرة التركية ..

.
كما وصلني .. https://t.co/qncixBrPXg
وين ولد الملحه 
عــــــــاجل
رئيس اركان الجيش الكويتي الفريق الركن

Every single rumor in the training data belongs to one one single user.

In [15]:
a = train_rumors["tweet_id"].value_counts()
max(a)

1

In [16]:
a = dev_rumors["tweet_id"].value_counts()
max(a)

1

### 2. Relevence data

In [17]:
rumors = pd.concat([train_rumors , dev_rumors])
rumors.head(3)

,rumor_id,tweet_id,tweet_text,category
0,AuFIN_001,1427297428890398725,موسى أبو مرزوق الحمساوى يهنئ طالبان بفوزه بأفغ...,politics
1,AuFIN_002,1311033577543479297,مزيداً من الأنحدار نحو مستنقع التطبيع القذر..ح...,politics
2,AuFIN_003,1225883281700290560,بتوجيهات من هيثم بن طارق آل سعيد سلطان عمان اس...,politics


In [18]:
columns = ["rumor_id", "0", "user_id", "relevance"]
relevance_data = text_to_dataframe(path_relevance_judgments , columns).drop(["0"] , axis=1)
relevance_data.tail(3)

,rumor_id,user_id,relevance
1041,AuFIN_023,1186788943930499072,2
1042,AuFIN_102,2572368029,1
1043,AuFIN_104,827130813632376836,2


This data assigns a degree of relevence to each pair of a rumor and one of its corresponding authorities; 2 if the authority is highly relevant to the rumor (has higher priority to be contacted); 1 if she is relevant.

The number of authorities used to verify the rumors in the training and developpement data are:

In [19]:
relevance_data["user_id"].value_counts()

259372802              25
142264595              13
460210929              13
569792145              12
293466923              11
                       ..
1295065674994204672     1
2276640199              1
1335830366              1
19280428                1
827130813632376836      1
Name: user_id, Length: 526, dtype: int64

526 authorities has been conssulted to verify the rumors of the training and developpement data.

In [20]:
relevance_data[relevance_data["user_id"] == "259372802"]    

,rumor_id,user_id,relevance
63,AuFIN_135,259372802,2
67,AuFIN_103,259372802,2
71,AuFIN_046,259372802,1
84,AuFIN_013,259372802,1
274,AuFIN_137,259372802,1
301,AuFIN_076,259372802,1
307,AuFIN_041,259372802,2
339,AuFIN_020,259372802,1
367,AuFIN_133,259372802,2
418,AuFIN_128,259372802,1


In [25]:
rumors[rumors["rumor_id"] == "AuFIN_132"]

,rumor_id,tweet_id,tweet_text,category
11,AuFIN_132,1282634667326410755,🔴مشروع أكبر صرح طبي للكشف عن فيروس كورونا في س...,health


In [26]:
user_id = "259372802"  #copy the user_id of the authority
link = "https://twitter.com/intent/user?user_id="+user_id
link

'https://twitter.com/intent/user?user_id=259372802'

The authority with **user_id = 259372802** has been used to verify **25** rumors. 

In [27]:
relevance_data["relevance"].value_counts()

2    619
1    425
Name: relevance, dtype: int64

only pairs with relevance = 1 (425 times) or 2 (619 times) are reported in the relevance_data. Relevance = 0 is assumed for all pairs not appearing in the dataframe.

In [28]:
relevance_data["rumor_id"].value_counts()

AuFIN_106    22
AuFIN_076    19
AuFIN_098    18
AuFIN_069    17
AuFIN_100    17
             ..
AuFIN_081     1
AuFIN_118     1
AuFIN_014     1
AuFIN_146     1
AuFIN_104     1
Name: rumor_id, Length: 150, dtype: int64

The number of unique rumors in the relevence data are 150 (**120 train_rumors** and **30 dev_rumors**).


The rumor with **ID = AuFIN_106** can be verified by **22** authorities.

In [29]:
relevance_data[relevance_data["rumor_id"] == "AuFIN_106"]

,rumor_id,user_id,relevance
344,AuFIN_106,4816936572,2
345,AuFIN_106,1116034096818466816,1
346,AuFIN_106,570223846,1
347,AuFIN_106,246727008,2
348,AuFIN_106,844860412008980481,2
349,AuFIN_106,710569086296653824,1
350,AuFIN_106,891970979374694400,1
351,AuFIN_106,1099928406156816384,2
352,AuFIN_106,709085388468588544,1
353,AuFIN_106,717844112364404738,1


In [30]:
# Data represents the dataframe containing the rumor with id=AuFIN_023
data = train_rumors[train_rumors["rumor_id"] == "AuFIN_023"]
print_rumors(data)

بكل فخر صنع في مصر 🇪🇬 رامي السبيعي اول مصري واول عربي يفوز بمستر اولمبيا في التاريخ مبروك لمصر والعرب #بيج_رامي https://t.co/BzNmoR33A4


To extract the user IDs of the authorities that can be consulted to verify a specific rumor ID (e.g., **AuFIN_023**), execute the following instruction:

In [31]:
data = relevance_data[relevance_data["rumor_id"] == "AuFIN_023"]
data

,rumor_id,user_id,relevance
737,AuFIN_023,70407496,2
738,AuFIN_023,435697807,2
1041,AuFIN_023,1186788943930499072,2


The user_id in relevance_data belongs to the user_id of the twitter account of the auathority. To check the twitter accounts associated to these authorities, execute the following instruction, and use the provided link. For instance https://twitter.com/intent/user?user_id=1186788943930499072 .


In [32]:
user_id = "259372802"  #copy the user_id of the authority
link = "https://twitter.com/intent/user?user_id="+user_id
link

'https://twitter.com/intent/user?user_id=259372802'

### 3. Twitter Lists Metadata

In [37]:
# df_metadata.head(2)

In [38]:
# df_metadata.describe()

In [39]:
# df_metadata["list_id"].value_counts()

There are 1193 Twitter lists, each list is mentioned one time in a provided Json file. Meaning each list is mentionned 1000 times.

In [40]:
# owner_id = "480830188"  #copy the user_id of the authority
# link = "https://twitter.com/intent/user?user_id="+user_id
# link

In [36]:
pwd

'C:\\Users\\us\\Documents\\Fake_News_detection'

### 4. User metadata

In [41]:
path_user_metadata = "data\\subtask-5A-arabic\\users_metadata_part981.json"
user_metadata = read_json_file(path_user_metadata)
user_metadata = json_to_dataframe(user_metadata)
user_metadata.head(3)

In [42]:
user_metadata["verified"].value_counts()

In [90]:
user_metadata.iloc[0,4] , user_metadata.iloc[0,2]

('احفظ الله يحفظك ارتوت واتابع مايروق لي فقط ومرحبا بكم في ملفي الشخصي',
 '﮼احفظ،الله،يحفظك.. ❀﴿welcome to myٰ profile﴾❀ ﮼ارتوت،واتابع،مايروق،لي،فقط..')

In [91]:
user_metadata.iloc[394,4] , len(user_metadata.iloc[394,4])

('التعامل في سرية تامة وبدون معرفة الشخص الي تبي تهكره', 52)

In [92]:
list1 = user_metadata["user_id"].value_counts().index

In [114]:
path_user_metadata = "data\\subtask-5A-arabic\\users_metadata_part12.json"
user_metadata = read_json_file(path_user_metadata)
user_metadata = json_to_dataframe(user_metadata)
user_metadata.head(10)

,user_id,name,description,translated_name,translated_desc,following_count,followers_count,verified,lists_count,lists_ids,collected_Arabic_tweets_ids
0,4899309747,noor kadry,Actress and singer and main cast member at Sat...,None,بالعربي الممثلة والمغنية وعضو فريق التمثيل الر...,261,19966,1,9,"1442183636363759616,1432109252777177089,140404...","795725679878881280,999828979665293314,69827968..."
1,36034541,Kalimat Group,مجموعة نشر إماراتية لكتب باللغة العربية حائزة ...,None,مجموعة نشر إماراتية لكتب باللغة العربية حائزة ...,2588,10853,1,131,"1459107451677011969,1398410646941806600,139326...","1232204707193552896,1364570634760589313,856140..."
2,1280770519973990402,Binance MENA,حساب @Binance الرسمي باللغة العربية لمنطقة الش...,بينانس مينا,حساب الرسمي باللغة العربية لمنطقة الشرق الأوسط...,17,56312,1,246,"1465832299912732681,1465768733427777541,146576...","1312791818858303490,1398335007215603712,135443..."
3,971064073801650177,AS Arabia,الحساب الرسمي لصحيفة AS الإسبانية باللغة العربية.,العربية,الحساب الرسمي لصحيفة الإسبانية باللغة العربية كما,14,202773,1,474,"1464509982469246979,1462387677219758091,146235...","1424748563393925131,1401491241741832194,141198..."
4,1381882025888657409,OKEx Arabia,حساب @OKEx الرسمي باللغة العربية تابعونا للمشا...,None,حساب الرسمي باللغة العربية تابعونا للمشاركة في...,15,2977,1,10,"1431246475896541189,1426156323260059652,141832...","1434545657793196033,1400041722336317442,141418..."
5,199011992,Red Bull Kuwait,@RedBullKuwait الموقع الرسمي لريد بُل - الكويت,ريد بل الكويت,الموقع الرسمي لريد بل الكويت,1075,18306,1,16,"1131852181898973185,1008614760835928064,921334...","473198497550704640,491864234729684992,79709196..."
6,1448217636,MBC.Net,حساب الموقع الرسمي لمجموعة \r\n@MBC_Group,ام بي سي نت,حساب الموقع الرسمي لمجموعة مجموعة إم بي سي,17,1643990,1,378,"1462494820367310860,1458662627232694272,145701...","1367126277090598913,1369659892487909382,130289..."
7,378517291,د. خالد الحسن,دكتوراه في السنة وعلومها من الجامعة الإسلامية ...,د خالد الحسن,دكتوراه في السنة وعلومها من الجامعة الإسلامية ...,247,14581,1,20,"1179624637677875200,1131852213578551296,110120...","882093102437150720,736104981812674560,13996931..."
8,535864454,محمد التوم,صانع محتوى | أعتز بوطني🇸🇦 | مقدم برنامج #مع_ال...,محمد التوم,صانع محتوى أعتز بوطني مقدم برنامج مع التوم عضو...,607,89047,1,41,"1431962067595112452,1415826874396184578,139162...","962743824677208065,1182996209943961602,9624716..."
9,2442783409,محمد عبدالصمد القرشي,عضو مجلس منطقة مكة المكرمة، عضو مجلس إدارة هيئ...,محمد عبدالصمد القرشي,عضو مجلس منطقة مكة المكرمة عضو مجلس إدارة هيئة...,90,151218,1,58,"1431962067595112452,1409997236113715202,113185...","910867277943382017,517599025483100160,46194790..."


In [113]:
user_metadata["verified"].value_counts()

1    396
Name: verified, dtype: int64

In [94]:
list2 = user_metadata["user_id"].value_counts().index

In [98]:
def list_diff(list1, list2):
    return [item for item in list1 if item not in list2]

len(list_diff(list1, list2))

395

In [99]:
user_metadata[user_metadata["user_id"] == 293466923]

,user_id,name,description,translated_name,translated_desc,following_count,followers_count,verified,lists_count,lists_ids,collected_Arabic_tweets_ids


If you want the link of a tweet_id from user_id, just copy the corresponding IDs following this instruction. 

In [104]:
user_id , tweet_id = "36034541" , "1232204707193552896"  #copy the user_id and tweet_id
link = "https://twitter.com/" + user_id + "/status/" + tweet_id
link

'https://twitter.com/36034541/status/1232204707193552896'